## 1) Import libraries 

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

## 2) Model


In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input -? OUtput? RF 26
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1) # 24
        self.pool1 = nn.MaxPool2d(2, 2)
        self.pintconv1 = nn.Conv2d(in_channels=64, out_channels=16, kernel_size=1)
        self.conv3 = nn.Conv2d(16, 32, 3, padding=1) # 12
        self.conv4 = nn.Conv2d(32, 64, 3, padding=1) # 10
        self.pool2 = nn.MaxPool2d(2, 2)
        self.pintconv2 = nn.Conv2d(in_channels=64, out_channels=16, kernel_size=1)
        self.conv5 = nn.Conv2d(16, 32, 3) # 5
        self.conv6 = nn.Conv2d(32, 64, 3) # 3
        self.conv7 = nn.Conv2d(64, 10, 3) # 1

        self.fc1 = nn.Linear(10+10, 128)                  #Concatenate two inputs 
        self.fc2 = nn.Linear(128, 18) # 0 <= sum <= 18

    def forward(self, img, random_number):
       
        #Image CNN
        img = self.pool1(F.relu(self.conv2(F.relu(self.conv1(img)))))
        img = F.relu(self.pintconv1(img))
        img = self.pool2(F.relu(self.conv4(F.relu(self.conv3(img)))))
        img = F.relu(self.pintconv2(img))
        img = F.relu(self.conv6(F.relu(self.conv5(img))))
        img = self.conv7(img)
        img = img.view(-1, 10)

        #Concatenate last layer of image and one hot encoding of random number   
        sum = torch.cat((img, random_number), dim=1)    

        #Sum 
        sum = F.relu(self.fc1(sum))
        sum = self.fc2(sum)  

        return F.log_softmax(img) , F.log_softmax(sum)

In [3]:
F.one_hot(torch.tensor(5), num_classes=10).shape

torch.Size([10])

## 3) Run on GPU if available

In [4]:
#!pip install torchsummary
#from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
#summary(model, [(1, 28, 28),(10)])

In [5]:
device

device(type='cuda')

## 4) Model graph

In [6]:
print(model)

Net(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (pintconv1): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1))
  (conv3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (pintconv2): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1))
  (conv5): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv7): Conv2d(64, 10, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=20, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=18, bias=True)
)


## 5) Custom Data Preparation

#### 5.1) download Mnist and create random number array

In [8]:
#set normalization values of mnist
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

#download train and test dataset 
trainset = datasets.MNIST(root='./data', download=True, train=True, transform=transform)
testset = datasets.MNIST(root='./data', download=True, train=False, transform=transform)

#create random numbers of the size of train and test dataset
#torch.randint is used as it generate random numbers uniformally  
randomnumtrainset = torch.randint(0, 9, (len(trainset),))
randomnumtestset = torch.randint(0, 9, (len(testset),))


Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw




Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw




Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


#### 5.2) Prepare Custom Dataset
getitem returns (mnist_image, mnist_label, random_number, sum_of_mnist_label_and_random_number)

In [9]:
from torch.utils.data import Dataset

class MNISTRandCombinedDataset(Dataset):
    def __init__(self, mnist,random_numbers):
        self.mnist = mnist
        self.random_numbers = random_numbers

    def __len__(self):
        return len(self.mnist)

    def __getitem__(self, idx):
        image, label = self.mnist[idx]
        random_number = self.random_numbers[idx].item()
        sum_label = label + random_number
        return image, label, F.one_hot(torch.tensor(random_number), num_classes=10), sum_label

#### 5.3) Prepare Dataloader

In [10]:

torch.manual_seed(1)
batch_size = 32
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
trainsetcombination = MNISTRandCombinedDataset(trainset,randomnumtrainset)
train_loader = torch.utils.data.DataLoader(trainsetcombination,
                                               batch_size=batch_size,
                                               shuffle=True, **kwargs
                                               )
testsetcombination = MNISTRandCombinedDataset(testset,randomnumtestset)
test_loader = torch.utils.data.DataLoader(testsetcombination,
                                               batch_size=batch_size,
                                               shuffle=False, **kwargs
                                               )


## 6) Defining taining and testing functions

In [11]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()   
    pbar = tqdm(train_loader)
    for batch_idx, (data, target, randnum, sumval) in enumerate(pbar):
        data, target, randnum, sumval = data.to(device), target.to(device), randnum.to(device), sumval.to(device) 
        optimizer.zero_grad()
        mnist_out, sum_out = model(data, randnum)
        loss = (F.nll_loss(mnist_out, target) + F.nll_loss(sum_out, sumval))/2
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')
    


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    mnist_correct = 0
    sum_correct = 0

    with torch.no_grad():
        for data, target, randnum, sumval in test_loader:
            data, target, randnum, sumval = data.to(device), target.to(device), randnum.to(device), sumval.to(device) 
            mnist_out, sum_out = model(data, randnum)
            
            # sum up batch loss
            test_loss += (F.nll_loss(mnist_out, target, reduction='sum').item() + F.nll_loss(sum_out, sumval, reduction='sum').item())/2 
            
            # get the index of max log-probability
            mnist_pred = mnist_out.argmax(dim=1, keepdim=True)
            sum_pred = sum_out.argmax(dim=1, keepdim=True)

            # increment the correct prediction count if pred is correct
            mnist_correct += mnist_pred.eq(target.view_as(mnist_pred)).sum().item()
            sum_correct += sum_pred.eq(sumval.view_as(sum_pred)).sum().item()
    
    print('\nTest set: Average loss: {:.4f}, Mnist_Accuracy: {}/{} ({:.0f}%), Sum_Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, 
        mnist_correct, len(test_loader.dataset),
        100. * mnist_correct / len(test_loader.dataset),
        sum_correct, len(test_loader.dataset),
        100. * sum_correct / len(test_loader.dataset),
        ))

## 7) Model Training

In [12]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 11):
    print(f"\nEpoch {epoch}")
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/1875 [00:00<?, ?it/s]


Epoch 1


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  0%|          | 0/1875 [00:00<?, ?it/s]


Test set: Average loss: 7155.6940, Mnist_Accuracy: 9678/10000 (97%), Sum_Accuracy: 6973/10000 (70%)


Epoch 2


  0%|          | 0/1875 [00:00<?, ?it/s]

...


Test set: Average loss: 282.7241, Mnist_Accuracy: 9929/10000 (99%), Sum_Accuracy: 9925/10000 (99%)


Epoch 9


  0%|          | 0/1875 [00:00<?, ?it/s]


Test set: Average loss: 430.1920, Mnist_Accuracy: 9902/10000 (99%), Sum_Accuracy: 9894/10000 (99%)


Epoch 10


loss=0.00295666023157537 batch_id=1874: 100%|██████████| 1875/1875 [00:26<00:00, 70.96it/s]



Test set: Average loss: 279.5039, Mnist_Accuracy: 9932/10000 (99%), Sum_Accuracy: 9920/10000 (99%)

